In [1]:
import mediapipe as mp
import tensorflow as tf
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from xml.etree import ElementTree

import os
import glob

2023-10-11 12:36:00.531776: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-11 12:36:00.671048: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-11 12:36:01.327655: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def get_image_from_folder(image_foler, basewidth=512):
    image_files = glob.glob(os.path.join(image_foler, "*", "*.jpg"))

    image_type = [i.split("/")[-2] for i in image_files]

    image_dict = []
    for image_file in image_files:
        img = Image.open(image_file)
        img = img.convert("RGB")
        
        wpercent = basewidth / img.size[0]
        hsize = int(img.size[1] * wpercent)
        img = img.resize((basewidth, hsize), Image.Resampling.LANCZOS)

        image_dict.append(np.array(img))

    return image_dict, image_type

In [3]:
image_data_1, image_type_data_1 = get_image_from_folder("TK")

In [4]:
def get_image(xml_path, image_path, basewidth=512):
    xml_files = glob.glob(os.path.join(xml_path, "*.xml"))

    image_files = []
    image_type = []
    for xml_file in xml_files:
        tree = ElementTree.parse(xml_file)
        root = tree.getroot()

        image_files.append(os.path.join(image_path, root.find('./filename').text))
        image_type.append(root.findall('.//name')[0].text)

    image_dict = []
    for image_file in image_files:
        img = Image.open(image_file)
        img = img.convert("RGB")

        wpercent = basewidth / img.size[0]
        hsize = int(img.size[1] * wpercent)
        img = img.resize((basewidth, hsize), Image.Resampling.LANCZOS)

        image_dict.append(np.array(img))

    return image_dict, image_type


In [5]:
image_data_2, image_type_data_2 = get_image("mystroke-PascalVOC-export/Annotations", "mystroke-PascalVOC-export/JPEGImages")

In [6]:
image_data_2.extend(image_data_1)
image_type_data_2.extend(image_type_data_1)

In [21]:
def get_hand_data(images, images_type=None):
    hands = mp.solutions.hands.Hands()

    new_images_type = []
    images_landmark = []
    for i, (img, img_type) in enumerate(zip(images, images_type)):
        hands_result = hands.process(img)
        hand_landmark = hands_result.multi_hand_landmarks
        if hand_landmark is None:
            if images_type is None:
                raise ValueError(f"all images needs to contain hand (at {i})")
            else:
                print(f"hand not detect {img_type}")
                continue

        landmark_list = []
        for landmark in hand_landmark[0].landmark:
            landmark_list.append(np.array([landmark.x, landmark.y, landmark.z]))

        landmark_array = np.array(landmark_list)

        images_landmark.append(landmark_array)
        new_images_type.append(img_type)

    return images_landmark, new_images_type
        

In [22]:
landmark_data, image_type_data_new = get_hand_data(image_data_2, image_type_data_2)

hand not detect RE
hand not detect HC
hand not detect HC
hand not detect WF
hand not detect WF
hand not detect WF
hand not detect WF
hand not detect UD
hand not detect UD
hand not detect UD
hand not detect UD
hand not detect UD
hand not detect UD
hand not detect UD
hand not detect UD
hand not detect UD
hand not detect UD
hand not detect UD
hand not detect UD
hand not detect UD
hand not detect UD
hand not detect UD
hand not detect UD
hand not detect TI
hand not detect FL
hand not detect FL
hand not detect HC
hand not detect TI
hand not detect UD
hand not detect WF


In [23]:
print("old ", len(image_type_data_2))
print(set(image_type_data_2))
print("new ", len(image_type_data_new))
print(set(image_type_data_new))

old  563
{'UD', 'WF', 'TI', 'RE', 'HC', 'FL', 'HO'}
new  533
{'UD', 'WF', 'TI', 'RE', 'HC', 'FL', 'HO'}


In [24]:
num_classes = len(set(image_type_data_new))

In [25]:
class_names = ['UD', 'HO', 'HC', 'TI', 'WF', 'FL', 'RE']

In [26]:
def one_hot(images_type, num_classes=7, classes=class_names):
    data_one_hot = []
    for img_type in images_type:
        one_hot_list = [0] * num_classes
        
        class_int = classes.index(img_type)
        
        one_hot_list[class_int] = 1

        data_one_hot.append(one_hot_list)

    return data_one_hot


In [27]:
image_type_one_hot_data = one_hot(image_type_data_new)

In [28]:
x_data = tf.data.Dataset.from_tensor_slices(landmark_data)
y_data = tf.data.Dataset.from_tensor_slices(image_type_one_hot_data)

dataset = tf.data.Dataset.zip((x_data, y_data))

In [29]:
dataset = dataset.shuffle(dataset.cardinality())

In [30]:
split = int(int(dataset.cardinality()) * 0.8) 

data_train = dataset.take(split)
data_val = dataset.skip(split)

val_split = int(int(data_val.cardinality()) * 0.6) 

data_test = data_val.skip(val_split)
data_val = data_val.take(val_split)

In [31]:
print(data_train.cardinality())
print(data_val.cardinality())
print(data_test.cardinality())

tf.Tensor(426, shape=(), dtype=int64)
tf.Tensor(64, shape=(), dtype=int64)
tf.Tensor(43, shape=(), dtype=int64)


In [32]:
data_train = data_train.batch(16)
data_val = data_val.batch(16)
data_test = data_test.batch(16)

In [33]:
AUTOTUNE = tf.data.AUTOTUNE

data_train = data_train.prefetch(AUTOTUNE)
data_val = data_val.prefetch(AUTOTUNE)
data_test = data_test.prefetch(AUTOTUNE)

In [34]:
class MyModel(tf.keras.Model):
    def __init__(self, num_classes=7):
        super().__init__()

        self.conv_1 = tf.keras.layers.Conv1D(8, 3, activation="mish")
        self.conv_2 = tf.keras.layers.Conv1D(16, 3, activation="mish")
        self.conv_3 = tf.keras.layers.Conv1D(32, 3, activation="mish")

        self.flatten = tf.keras.layers.Flatten()

        self.dense_1 = tf.keras.layers.Dense(64, activation="mish")
        self.dense_2 = tf.keras.layers.Dense(64, activation="mish")
        self.dense_3 = tf.keras.layers.Dense(64, activation="mish")
        self.dense_4 = tf.keras.layers.Dense(64, activation="mish")

        self.dense_last = tf.keras.layers.Dense(num_classes)

    def call(self, inputs):
        x = inputs

        x = self.conv_1(x)
        x = self.conv_2(x)
        x = self.conv_3(x)

        x = self.flatten(x)

        x = self.dense_1(x)
        x = self.dense_2(x)
        x = self.dense_3(x)
        x = self.dense_4(x)

        return  self.dense_last(x)


In [35]:
model = MyModel(num_classes)

model.compile(optimizer=tf.keras.optimizers.Lion(),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=[
                  tf.keras.metrics.CategoricalAccuracy(),
                  tf.keras.metrics.TopKCategoricalAccuracy(2),
                  ],
             )

In [36]:
checkpoint_filepath = "/tmp/checkpoint"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_filepath,
    monitor="val_categorical_accuracy",
    save_best_only=True,
    save_weights_only=True,
)

In [37]:
model.fit(
    data_train,
    epochs=100,
    validation_data=data_val,
    callbacks=[checkpoint_callback]
)

Epoch 1/100


2023-10-11 12:40:23.591940: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-10-11 12:40:24.610208: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-10-11 12:40:24.620722: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fa566ca5e00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-11 12:40:24.620742: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2023-10-11 12:40:24.639996: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-11 12:40:24.742251: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the p

27/27 [==============================] - 5s 21ms/step - loss: 1.9455 - categorical_accuracy: 0.1502 - top_k_categorical_accuracy: 0.2981 - val_loss: 1.9423 - val_categorical_accuracy: 0.1719 - val_top_k_categorical_accuracy: 0.3750
Epoch 2/100
27/27 [==============================] - 0s 6ms/step - loss: 1.9437 - categorical_accuracy: 0.1315 - top_k_categorical_accuracy: 0.3333 - val_loss: 1.9426 - val_categorical_accuracy: 0.1250 - val_top_k_categorical_accuracy: 0.2969
Epoch 3/100
27/27 [==============================] - 0s 6ms/step - loss: 1.9405 - categorical_accuracy: 0.1690 - top_k_categorical_accuracy: 0.3638 - val_loss: 1.9357 - val_categorical_accuracy: 0.1250 - val_top_k_categorical_accuracy: 0.3906
Epoch 4/100
27/27 [==============================] - 0s 7ms/step - loss: 1.9338 - categorical_accuracy: 0.2042 - top_k_categorical_accuracy: 0.4178 - val_loss: 1.9319 - val_categorical_accuracy: 0.2031 - val_top_k_categorical_accuracy: 0.4062
Epoch 5/100
27/27 [====================

In [38]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             multiple                  80        
                                                                 
 conv1d_1 (Conv1D)           multiple                  400       
                                                                 
 conv1d_2 (Conv1D)           multiple                  1568      
                                                                 
 flatten (Flatten)           multiple                  0         
                                                                 
 dense (Dense)               multiple                  30784     
                                                                 
 dense_1 (Dense)             multiple                  4160      
                                                                 
 dense_2 (Dense)             multiple                  416

In [39]:
model.load_weights(checkpoint_filepath)
_, accuracy, top_2_accuracy = model.evaluate(data_test)
print(f"Test accuracy: {round(accuracy * 100, 2)}%")
print(f"Test top 2 accuracy: {round(top_2_accuracy * 100, 2)}%")

3/3 [==============================] - 0s 46ms/step - loss: 0.1275 - categorical_accuracy: 0.9302 - top_k_categorical_accuracy: 1.0000
Test accuracy: 93.02%
Test top 2 accuracy: 100.0%


In [40]:
export_model = tf.keras.Sequential([
    model,
    tf.keras.layers.Activation('softmax')
])

export_model.compile(optimizer=tf.keras.optimizers.Lion(),
                     loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                     metrics=[
                             tf.keras.metrics.CategoricalAccuracy(),
                             tf.keras.metrics.TopKCategoricalAccuracy(2)
                             ]
                    )

_, accuracy_2, top_2_accuracy_2 = export_model.evaluate(data_test)
print(f"Test accuracy: {round(accuracy_2 * 100, 2)}%")
print(f"Test top 2 accuracy: {round(top_2_accuracy_2 * 100, 2)}%")

3/3 [==============================] - 0s 5ms/step - loss: 0.1300 - categorical_accuracy: 0.9535 - top_k_categorical_accuracy: 1.0000
Test accuracy: 95.35%
Test top 2 accuracy: 100.0%


In [41]:
class ExportingModel(tf.Module):
  def __init__(self, cnn_model, label, input_shape=(21, 3)):
    self.cnn_model = cnn_model
    self.input_shape = input_shape
    self.label = label

  def __call__(self, input_tensor):
    assert input_tensor.shape == self.input_shape

    prediction = self.cnn_model(tf.expand_dims(input_tensor, axis=0))

    return prediction, self.label

In [42]:
test = ExportingModel(export_model, class_names)

In [43]:
class ExportModel(tf.Module):
  def __init__(self, model):
    self.model = model

  @tf.function(input_signature=[tf.TensorSpec(shape=(21, 3), dtype=tf.float32)])
  def __call__(self, input_array):
    (prediction, label) = self.model(input_array)

    return prediction, label

In [45]:
export_model = ExportModel(test)

tf.saved_model.save(export_model, export_dir=f'hand_mediapipe_85-95_(21_2)')

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3, 3, 8), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fa9d0199a50>, 140367317452640), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3, 3, 8), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fa9d0199a50>, 140367317452640), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(8,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fa9bc1daad0>, 140366981679744), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(8,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fa9bc1daad0>, 140366981679744), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3, 8, 16), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fa9bc1eaa50>, 140367317618080), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(64, 7), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fa9946520d0>, 140367317595072), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(7,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fa9bc1c4090>, 140367317596672), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(7,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fa9bc1c4090>, 140367317596672), {}).


INFO:tensorflow:Assets written to: hand_mediapipe_85-95_(21_2)/assets


INFO:tensorflow:Assets written to: hand_mediapipe_85-95_(21_2)/assets
